In [1]:
import numpy as np
import mdtraj as md
import matplotlib.pyplot as plt
from Bio.SVDSuperimposer import SVDSuperimposer
import nglview as nv

# # Path: pymdna/__init__.py, prototype of the package and atomic is not properly referenced in the package at genertors.py now I just explicitly define the path loction
# import pymdna as mdna
import sys
sys.path.append('/Users/thor/surfdrive/Projects/pymdna/')
import pymdna as mdna 
import copy
%load_ext autoreload
%autoreload 2

joblib is not installed. Falling back to sequential computation.


In [2]:
traj = md.load('/Users/thor/surfdrive/Data/h-ns/BacterialChromatin/FI/0_k/2_ApT/dry_0.pdb')
traj.remove_solvent(inplace=True)
traj = traj.atom_slice(traj.topology.select('not protein'))[0]
sequence = mdna.utils.get_sequence_letters(traj)
print(sequence)
mutations = {0: 'A', 6: 'T'}
mutant = mdna.Mutate(traj, mutations, complementary=True)
new_sequence = mdna.utils.get_sequence_letters(mutant.traj)
print(new_sequence)

['G', 'C', 'G', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'G', 'C']
['G', 'C', 'G', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'G', 'C']


In [4]:
new_traj = mutant.mutant_traj
view = nv.show_mdtraj(new_traj)
view.clear()
view.add_representation('licorice', selection=':A',color='cornflowerblue')
view.add_representation('licorice', selection=':B',color='coral')
view

NGLWidget()

In [ ]:
# Load dna for traj
# Find residues that needs to be mutated
# For each residue:
    # Determine the atoms that need to be replace
    # Collect the atoms that need to be inserted
    # Define for both residues a local reference frame
    # Transform the atoms to the local reference frame
    # Replace the atoms in the original structure (xyz)
    # Update the topology


# sug_atoms = {"C1'", "C2'", "C3'", "C4'", "O4'"}
# backbone = {"P", "C5'", "C3'", "O3'", "O5'"}

# nucl_res = {
#         'ADE', 'URA', 'CYT', 'GUA', 'THY', 'DA', 'DC', 'DG', 'DT', 'RA',
#         'RU', 'RG', 'RC', 'A', 'T', 'U', 'C', 'G',
#         'DA5', 'DC5', 'DG5', 'DT5',
#         'DA3', 'DC3', 'DG3', 'DT3',
#         'RA5', 'RU5', 'RG5', 'RC5',
#         'RA3', 'RU3', 'RG3', 'RC3'
#     }


In [3]:
def get_base_indices(traj, resid=0):

    # Define the atoms that belong to the nucleotide
    base_atoms = {'N9', 'N7', 'C8', 'C5', 'C4', 'N3', 'C2', 'N1', 
                  'C6', 'C7','O6', 'N2', 'N6', 'O2', 'N4', 'O4', 'C5M',
                  'H1','H2','H21','H22','H3','H41','H42','H5','H6','H61','H62','H71','H72','H73','H8'}
    # Select atoms that belong to the specified residue
    indices = traj.top.select(f'resid {resid}')
    offset = indices[0]  # Save the initial index of the residue
   
    # Create a subtrajectory containing only the specified residue
    subtraj = traj.atom_slice(indices)

    # Select the atoms that belong to the nucleotide
    sub_indices = subtraj.top.select(f'name {" ".join(base_atoms)}')
    # Return the indices of the atoms that belong to the nucleotide
    return sub_indices + offset


def make_complementary_mutations(traj, mutations, base_pair_map):
    
    # Get the basepair dictionary of the trajectory
    basepair_dict = mdna.utils.get_base_pair_dict(traj)

    # Iterate over a static list of dictionary items to avoid RuntimeError
    for idx, base in list(mutations.items()):
        
        # Get the complementary base
        comp_base = basepair_dict[traj.top._residues[idx]]
        comp_mutant = base_pair_map[base]
    
        # Update mutations with the complementary base's mutation
        mutations[comp_base.index] = comp_mutant
        
    return mutations


def update_mutant_topology(traj, target_indices, mutant_indices, base, resid, mutation_traj):

    # Delete the target atoms from the original structure (top)
    # Sort and reverse the indices to avoid index errors (because index gets updated after each deletion)
    for index in np.sort(target_indices)[::-1]:
        traj.top.delete_atom_by_index(index)

    # Calculate the offset to insert the new atoms
    offset = target_indices[0]
    # Insert the new atoms in the original structure (top)
    for idx,index in enumerate(mutant_indices):
        
        # Atom to insert
        atom = mutation_traj.top.atom(index)

        # Find offset atom in the residue
        offset_atom = traj.top._atoms[offset]
        for i,at in enumerate(traj.top._residues[resid]._atoms):
            if at.index == offset_atom.index:
                insert_id = i
                break

        # Insert the atom in the topology
        traj.top.insert_atom(name=atom.name, element=atom.element, residue=traj.top._residues[resid], 
                            index=traj.top._atoms[offset].index+idx, rindex=insert_id+idx)
    
    # Update the residue name
    traj.top._residues[resid].name = 'D'+base
    return traj


def get_base_transformation(mutant_reference,target_reference):

    # Collect the reference information of the mutation
    mutation_origin = mutant_reference.b_R[0]
    D = mutant_reference.b_D[0]
    L = mutant_reference.b_L[0]
    N = mutant_reference.b_N[0]
    mutation_basis = np.array([D,L,N])

    # Collect the reference information of the target to mutate
    target_ref = mdna.ReferenceBase(target_reference)
    target_origin = target_ref.b_R[0]
    target_basis = np.array([target_ref.b_D[0],target_ref.b_L[0],target_ref.b_N[0]])

    # Calculate the transformation 
    rot = np.linalg.solve(target_basis,mutation_basis)
    trans = target_origin - mutation_origin
    return rot, trans

def update_mutant_topology(traj, target_indices, mutant_indices, base, resid, mutation_traj):
    print(target_indices, mutant_indices)
    print(base, resid, mutation_traj)

    # Delete the target atoms from the original structure (top)
    # Sort and reverse the indices to avoid index errors (because index gets updated after each deletion)
    for index in np.sort(target_indices)[::-1]:
        traj.top.delete_atom_by_index(index)

    # Calculate the offset to insert the new atoms
    offset = target_indices[0]
    print('offset', offset)

    # Insert the new atoms in the original structure (top)
    for idx,index in enumerate(mutant_indices):
        
        # Atom to insert
        atom = mutation_traj.top.atom(index)

        # Find offset atom in the residue
        offset_atom = traj.top._atoms[offset]
        print('offset_atom',offset_atom)
        for i,at in enumerate(traj.top._residues[resid]._atoms):
            if at.index == offset_atom.index:
                insert_id = i
                break
        print(atom.name, atom.element, traj.top._residues[resid], traj.top._atoms[offset].index+idx, insert_id+idx)
        # Insert the atom in the topology
        traj.top.insert_atom(name=atom.name, element=atom.element, residue=traj.top._residues[resid], 
                            index=traj.top._atoms[offset].index+idx, rindex=insert_id+idx)
        print('\n')
    # Update the residue name
    traj.top._residues[resid].name = 'D'+base
    return traj
    
def apply_mutations(traj, mutations, base_pair_map):

    # Make a copy of the original trajectory
    traj = copy.deepcopy(traj)
    
    # This comes now from sequence generator.py, either move this to somewhere else such that it is accessible everywhere
    reference_bases = {base: md.load_pdb(f'/Users/thor/surfdrive/Projects/pymdna/pymdna/atomic/NDB96_{base}.pdb') for base in base_pair_map.keys()}
    reference_frames = {letter: mdna.ReferenceBase(t) for letter,t in reference_bases.items()}

    # For each residue that needs to be mutated
    for resid,base in mutations.items():

        # Get the mutant trajectory object
        mutation_traj = reference_bases[base] 

        # Get the indices of the atoms that need to be transformed
        mutant_indices = get_base_indices(mutation_traj, resid=0)
        target_indices = get_base_indices(traj, resid=resid)
        
        # Get the transformation for the local reference frames from the mutant to the target
        mutant_reference = reference_frames[base]
        target_reference = traj.atom_slice(traj.top.select(f'resid {resid}'))
        rot, trans = get_base_transformation(mutant_reference, target_reference)

        # Transform the mutant atoms to the local reference frame of the target
        mutant_xyz = mutation_traj.xyz[:,mutant_indices,:]
        new_xyz = np.dot(mutant_xyz, rot.T) + trans    

        # Get the original xyz coordinates
        xyz = traj.xyz 

        # Split the xyz in 2 pieces, one before the indices that need to be replaced, and the indices after the indices that need to be replaced
        xyz1 = xyz[:,:target_indices[0],:] 
        xyz2 = xyz[:,target_indices[-1]+1:,]

        # Update the topology
        traj = update_mutant_topology(traj, target_indices, mutant_indices, base, resid, mutation_traj)

        # Concatenate the new xyz with the original xyz
        xyz = np.concatenate([xyz1, new_xyz, xyz2], axis=1)
        traj.xyz = xyz

    # Return the mutated trajectory
    return traj

In [4]:
traj = md.load('/Users/thor/surfdrive/Data/h-ns/BacterialChromatin/FI/0_k/2_ApT/dry_0.pdb')
# traj = md.load('../biomolsim/dna.pdb')
traj.remove_solvent(inplace=True)
traj = traj.atom_slice(traj.topology.select('not protein'))[0]
# traj = traj.atom_slice(traj.topology.select('resid 0 23'))# and not element symbol H'))

# Create a DNA object
dna = mdna.NucleicFrames(traj)

# Define the base pair map and the complementary mutant map
base_pair_map = {'A': 'T', 'T': 'A', 'C': 'G', 'G': 'C'}#,'P':'T','M':'C','H':'T'}


# Get base frames from current DNA sequence
base_frames = dna.frames

# # Define the mutation to be performed
complementary = False
mutations = {0: 'A'}#, 6: 'T'}

if complementary:
    # Update dict with the complementary mutations
    mutations = make_complementary_mutations(traj, mutations, base_pair_map)

print(mutations)
sequence = mdna.utils.get_sequence_letters(traj)
sequence_pairs = mdna.utils.get_base_pair_letters(traj)
new_sequences = [mutations.get(idx, seq) for idx, seq in enumerate(sequence)]

print('WT',sequence)
# print(new_sequences)

# Apply the mutations
mutant_traj = apply_mutations(traj, mutations, base_pair_map)
    
new_sequence = mdna.utils.get_sequence_letters(mutant_traj)
print('M ',new_sequence)
view = nv.show_mdtraj(mutant_traj)
view.clear_representations()
view.add_representation('ball+stick', selection='all')
view


{0: 'A'}
WT ['G', 'C', 'G', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'G', 'C']
[10 11 12 13 14 15 16 17 18 19 20 21 22 23 24] [ 1  2  3  4  5  6  7  8  9 10]
A 0 <mdtraj.Trajectory with 1 frames, 11 atoms, 1 residues, without unitcells>
offset 10
offset_atom DG1-C3'
N9 nitrogen DG1 10 10


offset_atom DG1-N9
C8 carbon DG1 11 11


offset_atom DG1-N9
N7 nitrogen DG1 12 12


offset_atom DG1-N9
C5 carbon DG1 13 13


offset_atom DG1-N9
C6 carbon DG1 14 14


offset_atom DG1-N9
N6 nitrogen DG1 15 15


offset_atom DG1-N9
N1 nitrogen DG1 16 16


offset_atom DG1-N9
C2 carbon DG1 17 17


offset_atom DG1-N9
N3 nitrogen DG1 18 18


offset_atom DG1-N9
C4 carbon DG1 19 19


M  ['A', 'C', 'G', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'G', 'C']


NGLWidget()

In [ ]:
original = md.load('/Users/thor/surfdrive/Data/h-ns/BacterialChromatin/FI/0_k/2_ApT/dry_0.pdb')
mutant = md.load('mutant_test.pdb')

merged = original.stack(mutant)
view = nv.show_mdtraj(merged)
view.clear_representations()
view.add_representation('ball+stick', selection='not protein')
view

In [ ]:
# First collect xyz coordinates of the base pairs
xyz, indices = self.get_basepair_xyz(basepairs,idx)

# Apply the rotation and translation
new_xyz = np.dot(xyz,rot) + trans

# Set the new coordinates
self.traj.xyz[:,indices,:] = new_xyz



In [ ]:
ref_origin = reference_frames['G'].b_R
ref_basis = reference_frames['G'].basis
ref_traj = reference_bases['G'] 


traget_residue = traj.top._residues[5]
target_base = base_frames[traget_residue]
target_origin = target_base[0]
target_basis = target_base[1:4]

rot = np.linalg.solve(ref_basis, target_basis)
trans = ref_origin - target_origin

# find indices of atoms that need to be replaced
traj.top.select('resid 5 and name P')
# find indices of atoms that need to be inserted


In [ ]:
def get_base_indices(traj, resid=0):
    
    # Define the atoms that belong to the nucleotide
    base_atoms = {'N9', 'N7', 'C8', 'C5', 'C4', 'N3', 'C2', 'N1', 
                  'C6', 'C7','O6', 'N2', 'N6', 'O2', 'N4', 'O4', 'C5M',
                  'H1','H2','H21','H22','H3','H41','H42','H5','H6','H61','H62','H71','H72','H73','H8'}
    
    # Select atoms that belong to the specified residue
    indices = traj.top.select(f'resid {resid}')
    offset = indices[0]  # Save the initial index of the residue
    print(traj.top._residues[resid], indices)
    # Create a subtrajectory containing only the specified residue
    subtraj = traj.atom_slice(indices)

    # Select the atoms that belong to the nucleotide
    sub_indices = subtraj.top.select(f'name {" ".join(base_atoms)}')
    # Return the indices of the atoms that belong to the nucleotide
    return sub_indices + offset
    #return indices

selection = get_base_indices(traj, resid=6)

view = nv.show_mdtraj(traj.atom_slice(selection))
view

In [ ]:
import networkx as nx

def check_component(component):
    # Iterating over each node in the component
    for node in component:
        # Check if the node name is 'P'
        if node.name == 'P':
            return False  # Return False if a 'P' node is found
    return True  # Return True if no 'P' node is found

# KAK dit werkt niet for DT maar wel DC?!? 
def get_nucleotide_indices(traj, resid=0):

    # Select atoms that belong to the specified residue
    indices = traj.top.select(f'resid {resid}')
    offset = indices[0]  # Save the initial index of the residue

    # Create a subtrajectory containing only the specified residue
    subtraj = traj.atom_slice(traj.topology.select(f'resid {resid}'))
    subtraj.top.create_standard_bonds()
    # Convert the topology of the subtrajectory to a bond graph
    G = subtraj.top.to_bondgraph()

    # Identify the 'C1'' atom in the subtrajectory
    atoms = subtraj.top._atoms
    for at in atoms:
        if at.name == 'C1\'':
            C1 = at  # Store the 'C1'' atom
    print(C1)

    # Find neighbors of 'C1'' and identify the nitrogen atom connected to it
    neighbors = G[C1]
    print(neighbors)
    m = [n for n in neighbors if 'N' in n.name][0]
    # Remove the edge between 'C1'' and the nitrogen atom
    G.remove_edges_from([(C1, m)])

    # Find connected components in the modified graph
    connected_components = nx.connected_components(G)
    # Filter out components with more than one node
    large_components = [component for component in connected_components if len(component) > 1]
    # Identify the nucleotide component by using the check_component function
    nucleotide_component = [c for c in large_components if check_component(c)][0]

    # Return the indices of the atoms in the nucleotide component plus the offset
    return [node.index for node in nucleotide_component] + offset

# Example usage of the function

traj = reference_bases['T']
traj.top.create_standard_bonds()    
print(traj.top)

selection = get_nucleotide_indices(traj, resid=0)

view = nv.show_mdtraj(traj.atom_slice(selection))
view


In [ ]:
view = nv.show_mdtraj(reference_bases['T'])
view